In [1]:
debugging = True
IPTS = '19558'

# Description 

This notebook performs the normalization of a set of images (tiff or fits) by OBs. 

The steps are:
 - select your **sample images**
 - select your **ob images**
 - select your **df images** (optional)
 - all the data will be loaded
 - preview of the sample and ob
 - selection of a **background** region in the sample data
 - normalization is performed
 - export of the normalized data

# Python Import

In [2]:
from __code.__all import custom_style
custom_style.style()

In [3]:
from __code.normalization import NormalizationHandler

In [4]:
%matplotlib notebook

In [5]:
from __code import utilities, gui_widgets
working_dir = utilities.get_working_dir(ipts=IPTS, debugging=debugging)
print("Working dir: {}".format(working_dir))

Working dir: /Volumes/my_book_thunderbolt_duo/IPTS/IPTS_19558


# Select Sample Images 

In [6]:
o_norm_handler = NormalizationHandler(working_dir = working_dir)
o_norm_handler.select_images()

HTML(value='Please wait...')

VBox(children=(Label(value='Select sample images', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_19558', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .DS_Store                |     Fri Oct 27 09:55:29 2017', ' normalization            |     Directory', ' run_02                   |     Directory', ' run_02_normalization     |     Directory', ' run_08_OB                |     Directory', ' run_16                   |     Directory', ' run_17                   |     Directory', ' run_18                   |     Directory', ' run_19                   |     Directory', ' run_20                   |     Directory', ' run_22                   |     Directory'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

HTML(value='Please wait...')

VBox(children=(Label(value='Select sample images', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Users/j35/Desktop', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .DS_Store                                                                          |     Mon Oct 30 09:41:53 2017', ' .ipynb_checkpoints                                                                 |     Directory', ' .localized                                                                         |     Thu Feb 25 16:55:24 2016', ' 14-G01337_VENUS-trifold-20150312_A.pdf                                             |     Wed Oct 25 09:56:17 2017', ' 15_G00337A_SNS_Instruments_links.pdf                                               |     Tue Oct 24 10:44:44 2017', ' 703 Easy Street by The Collapsable Hearts Club ft. Jim Biano & Petra Haden.mp3     |     Fri Nov 11 16:04:30 2016', ' Application_form_IMAGING_2017_v2 alias                                             |     Tue Sep  5 16:16:09 2017', ' Application_form_IMAGING_2017_v2.doc                                               |     Wed Jul  5 08:45:12 2017', ' CG1D_jean.docx                                                                     |     Fri Apr 28 08:54:42 2017', ' Imaging_and_engineering_science_instruement_suite_review_2017.docx                 |     Wed Sep 20 13:52:19 2017', ' MD-PFO_30_1T_chi3_388.zip                                                          |     Wed Nov 16 11:13:39 2016', ' SHORT_Run19_normalized_bins_fit                                                    |     Directory', ' bin                                                                                |     Directory', ' conferencecall_TEMPLATE.pptx                                                       |     Fri Mar 31 11:51:23 2017', ' iBeatles                                                                           |     Directory', ' ipts_17099_notebook.png                                                            |     Fri Mar 10 09:29:20 2017', ' remove_me_squish                                                                   |     Directory', ' ~$160725 Art Couplage Imagerie_v2[2].docx                                          |     Tue Aug  2 09:20:41 2016', ' ~$17 April 10  Trentesaux FINAL 04-07-2017.docx                                    |     Fri Apr  7 13:22:35 2017'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

HTML(value='Please wait...')

VBox(children=(Label(value='Select sample images', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Users/j35/Desktop', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .DS_Store                                                                          |     Mon Oct 30 09:41:53 2017', ' .ipynb_checkpoints                                                                 |     Directory', ' .localized                                                                         |     Thu Feb 25 16:55:24 2016', ' 14-G01337_VENUS-trifold-20150312_A.pdf                                             |     Wed Oct 25 09:56:17 2017', ' 15_G00337A_SNS_Instruments_links.pdf                                               |     Tue Oct 24 10:44:44 2017', ' 703 Easy Street by The Collapsable Hearts Club ft. Jim Biano & Petra Haden.mp3     |     Fri Nov 11 16:04:30 2016', ' Application_form_IMAGING_2017_v2 alias                                             |     Tue Sep  5 16:16:09 2017', ' Application_form_IMAGING_2017_v2.doc                                               |     Wed Jul  5 08:45:12 2017', ' CG1D_jean.docx                                                                     |     Fri Apr 28 08:54:42 2017', ' Imaging_and_engineering_science_instruement_suite_review_2017.docx                 |     Wed Sep 20 13:52:19 2017', ' MD-PFO_30_1T_chi3_388.zip                                                          |     Wed Nov 16 11:13:39 2016', ' SHORT_Run19_normalized_bins_fit                                                    |     Directory', ' bin                                                                                |     Directory', ' conferencecall_TEMPLATE.pptx                                                       |     Fri Mar 31 11:51:23 2017', ' iBeatles                                                                           |     Directory', ' ipts_17099_notebook.png                                                            |     Fri Mar 10 09:29:20 2017', ' remove_me_squish                                                                   |     Directory', ' ~$160725 Art Couplage Imagerie_v2[2].docx                                          |     Tue Aug  2 09:20:41 2016', ' ~$17 April 10  Trentesaux FINAL 04-07-2017.docx                                    |     Fri Apr  7 13:22:35 2017'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

HTML(value='Please wait...')

VBox(children=(Label(value='Select sample images', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .DS_Store                   |     Wed May 18 10:02:30 2016', ' Macintosh HD                |     Directory', ' my_book_thunderbolt_duo     |     Directory'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

HTML(value='Please wait...')

VBox(children=(Label(value='Select sample images', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .DS_Store                        |     Wed Sep 13 09:17:07 2017', ' .DocumentRevisions-V100          |     Directory', ' .Spotlight-V100                  |     Directory', ' .TemporaryItems                  |     Directory', ' .Trashes                         |     Directory', ' .fseventsd                       |     Directory', ' AfterEffectCAche                 |     Directory', ' Documents                        |     Directory', ' Dropbox (ORNL)                   |     Directory', ' IPTS                             |     Directory', ' Music                            |     Directory', ' PyQtProjects                     |     Directory', ' Run32_Step1_AM_polymer           |     Directory', ' System                           |     Directory', ' Videos                           |     Directory', ' code                             |     Directory', ' conferences                      |     Directory', ' iBeatles                         |     Directory', ' iMovie Library.imovielibrary     |     Directory', ' nocesdor                         |     Directory', ' temporary                        |     Directory', ' wingIDE_projects                 |     Directory'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

# Select Open Beam (OB)

In [7]:
o_norm_handler.select_images(data_type='ob')

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

# Select Dark Field (DF) - Optional

In [ ]:
o_norm_handler.select_images(data_type='df')

# Loding data 

In [8]:
o_norm_handler.load_data()

A Jupyter Widget

A Jupyter Widget

# Preview Data

In [9]:
o_norm_handler.plot_images(data_type='sample')

A Jupyter Widget

In [10]:
o_norm_handler.plot_images(data_type='ob')

A Jupyter Widget

# Select Background Region 

In [10]:
o_norm_handler.with_or_without_roi()

A Jupyter Widget

In [13]:
o_norm_handler.select_sample_roi()

A Jupyter Widget

A Jupyter Widget

# Normalization

In [16]:
o_norm_handler.run_normalization()

# Export 

In [17]:
o_norm_handler.select_export_folder()

A Jupyter Widget

A Jupyter Widget

In [18]:
o_norm_handler.export()

A Jupyter Widget